In [1]:
import datetime as dt
import requests
from bs4 import BeautifulSoup

In [2]:
url = "https://blog.hossam.kr"

In [3]:
session = requests.Session()

session.headers.update({
    "Referer": "",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
})


In [6]:
def getContents(session,url):
    try:
        r = session.get(url)

        if r.status_code != 200:
            msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
            raise Exception(msg)
    except Exception as e:
        print("접속에 실패했습니다.")
        print(e)

    r.encoding = "utf-8"
    soup = BeautifulSoup(r.text)
    post = soup.select(".post")
    print("추출한 컨텐츠의 수:", len(post))

    mydata = []

    mydata = []

    for i, v in enumerate(post):
        #print(v)

        entryTitle = v.select(".entry-title a")
        #print(entryTitle)

        title = entryTitle[0].text.strip()
        #print(title)

        if "href" in entryTitle[0].attrs:
            href = entryTitle[0].attrs['href']

            if url not in href:
                href = "%s%s" % (url, href)
        else:
            href = None
        #print(href)            
        published = v.select(".published")
        #print(published)

        datetime = published[0].attrs['datetime']
        #print(datetime)

        entryContent = v.select(".entry-content p:not(.read-more)")
        #print(entryContent)

        text = ""
        for j, w in enumerate(entryContent):
            text += " " + w.text.strip() if j > 0 else w.text.strip()
        #print(text)

        tagLinks = v.select(".tag-links a")
        #print(tagLinks) 

        for j, w in enumerate(tagLinks):
            tagLinks[j] = w.text.strip()

        #print(tagLinks)
        tags = ",".join(tagLinks)
        #print(tags)

        mydict = {
            "title": title,
            "href": href,
            "datetime": datetime,
            "text": text,
            "tags": tags
        }
        print(mydict)

        mydata.append(mydict)       

    return mydata

In [8]:
result = getContents(session, "https://blog.hossam.kr")

추출한 컨텐츠의 수: 5
{'title': '[Toy Project] 서버 가동 설정 수정', 'href': 'https://blog.hossam.kr/2023/08/21/Toy-%EC%84%9C%EB%B2%84_%EA%B0%80%EB%8F%99_%EC%84%A4%EC%A0%95_%EC%88%98%EC%A0%95/', 'datetime': '2023-08-21', 'text': '납품했던 웹 사이트에 대해 몇 가지 수정사항이 발생하였다. 이를 처리하는 과정에서 진행중인 토이프로젝트의 설정도 수정해야 할 부분이 발견되었다.', 'tags': 'Toy Project'}
{'title': '[Toy Project] Next.js와 Express 병합', 'href': 'https://blog.hossam.kr/2023/08/20/Toy-Next.js+Express-%EB%B3%91%ED%95%A9/', 'datetime': '2023-08-20', 'text': 'Backend와 Frontend가 각각 독립적인 시스템으로 구축되어 서로 연동하는 것이 이상적인 형태이겠지만 웹호스팅을 사용하는 경우 두개의 호스팅 계정을 만들지 않은 이상은 하나의 웹 서버안에서 Frontend와 Backend를 모두 처리해야 한다. 그렇기 때문에 우선적으로 구성해야 하는 환경은 Next.js에 백엔드 시스템을 병합하는 것이다. Next.js 자체적으로 API Route 기능을 제공하기는 하지만 백엔드 시스템으로 사용하기에는 다소 부족한 부분이 있기 때문에 Express를 병합하기로 하였다.', 'tags': 'Toy Project'}
{'title': '[Toy Project] 토이 프로젝트 시작', 'href': 'https://blog.hossam.kr/2023/08/17/Toy-%ED%86%A0%EC%9D%B4%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8-%EC%8B%9C%EC%9E%91/', 'datetime': '2023-08-17', 'text': '지난주

In [11]:
url = "https://blog.hossam.kr"
urlFmt = "%s/blog/page{pageNumber}/" % url
urlFmt

'https://blog.hossam.kr/blog/page{pageNumber}/'

In [12]:
maxPage = 22

In [18]:
blogPosts = []

for i in range(1, maxPage+1):
    targetUrl = urlFmt.format(pageNumber=i) if i>1 else url
    blogPosts += getContents(session, targetUrl)

print("수집된 전체 게시글 수: ", len(blogPosts))

blogPosts

추출한 컨텐츠의 수: 5
{'title': '[Toy Project] 서버 가동 설정 수정', 'href': 'https://blog.hossam.kr/2023/08/21/Toy-%EC%84%9C%EB%B2%84_%EA%B0%80%EB%8F%99_%EC%84%A4%EC%A0%95_%EC%88%98%EC%A0%95/', 'datetime': '2023-08-21', 'text': '납품했던 웹 사이트에 대해 몇 가지 수정사항이 발생하였다. 이를 처리하는 과정에서 진행중인 토이프로젝트의 설정도 수정해야 할 부분이 발견되었다.', 'tags': 'Toy Project'}
{'title': '[Toy Project] Next.js와 Express 병합', 'href': 'https://blog.hossam.kr/2023/08/20/Toy-Next.js+Express-%EB%B3%91%ED%95%A9/', 'datetime': '2023-08-20', 'text': 'Backend와 Frontend가 각각 독립적인 시스템으로 구축되어 서로 연동하는 것이 이상적인 형태이겠지만 웹호스팅을 사용하는 경우 두개의 호스팅 계정을 만들지 않은 이상은 하나의 웹 서버안에서 Frontend와 Backend를 모두 처리해야 한다. 그렇기 때문에 우선적으로 구성해야 하는 환경은 Next.js에 백엔드 시스템을 병합하는 것이다. Next.js 자체적으로 API Route 기능을 제공하기는 하지만 백엔드 시스템으로 사용하기에는 다소 부족한 부분이 있기 때문에 Express를 병합하기로 하였다.', 'tags': 'Toy Project'}
{'title': '[Toy Project] 토이 프로젝트 시작', 'href': 'https://blog.hossam.kr/2023/08/17/Toy-%ED%86%A0%EC%9D%B4%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8-%EC%8B%9C%EC%9E%91/', 'datetime': '2023-08-17', 'text': '지난주

[{'title': '[Toy Project] 서버 가동 설정 수정',
  'href': 'https://blog.hossam.kr/2023/08/21/Toy-%EC%84%9C%EB%B2%84_%EA%B0%80%EB%8F%99_%EC%84%A4%EC%A0%95_%EC%88%98%EC%A0%95/',
  'datetime': '2023-08-21',
  'text': '납품했던 웹 사이트에 대해 몇 가지 수정사항이 발생하였다. 이를 처리하는 과정에서 진행중인 토이프로젝트의 설정도 수정해야 할 부분이 발견되었다.',
  'tags': 'Toy Project'},
 {'title': '[Toy Project] Next.js와 Express 병합',
  'href': 'https://blog.hossam.kr/2023/08/20/Toy-Next.js+Express-%EB%B3%91%ED%95%A9/',
  'datetime': '2023-08-20',
  'text': 'Backend와 Frontend가 각각 독립적인 시스템으로 구축되어 서로 연동하는 것이 이상적인 형태이겠지만 웹호스팅을 사용하는 경우 두개의 호스팅 계정을 만들지 않은 이상은 하나의 웹 서버안에서 Frontend와 Backend를 모두 처리해야 한다. 그렇기 때문에 우선적으로 구성해야 하는 환경은 Next.js에 백엔드 시스템을 병합하는 것이다. Next.js 자체적으로 API Route 기능을 제공하기는 하지만 백엔드 시스템으로 사용하기에는 다소 부족한 부분이 있기 때문에 Express를 병합하기로 하였다.',
  'tags': 'Toy Project'},
 {'title': '[Toy Project] 토이 프로젝트 시작',
  'href': 'https://blog.hossam.kr/2023/08/17/Toy-%ED%86%A0%EC%9D%B4%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8-%EC%8B%9C%EC%9E%91/',
  'datetime': '2023-08-17',
 

In [21]:
fname = dt.datetime.now().strftime("블로그_글_수집_%y%m%d_%H%M%S.csv")

with open(fname, "w", encoding='utf-8') as f:
    for i, v in enumerate(blogPosts):
        if i == 0:
            title = "%s\n" % ",".join(list(v.keys()))
            f.write(title)

        content = list(v.values())

        for j, w in enumerate(content):
                content[j] = '"%s"' % w.replace('"','\\"')
        
        f.write("%s\n" % ",".join(content))